In [2]:
#Connect to GIS
from arcgis.gis import GIS
gis = GIS("home")

In [3]:
import requests
import pandas as pd
import zipfile
import os
import io
import arcpy

In [4]:
directory='C:\Mac\Home\Documents\ArcGIS\Lab2Data'
url = r"https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-12-05.las"
in_las_dataset = requests.get(url)
f= open(r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\4342-12-05.las", 'wb')
f.write(in_las_dataset.content)
f.close()

In [5]:
out_tin = 'C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\example.tin'
arcpy.ddd.LasDatasetToTin('C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\4342-12-05.las', out_tin, thinning_type= 'random', thinning_method='percent', thinning_value= 30)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\Lab2_Notebooks\\example.tin'>

In [6]:
arcpy.conversion.LasDatasetToRaster('C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\4342-12-05.las', 'las_dem')

ExecuteError: ERROR 000258: Output C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\Lab2_Notebooks.gdb\las_dem already exists
Cannot acquire a schema lock because of an existing lock.
Failed to execute (LasDatasetToRaster).


In [ ]:
zip_url='https://ftp.prism.oregonstate.edu/normals/monthly/4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip'
zip_response= requests.get(zip_url)
zip_path ='C:\Mac\Home\Documents\ArcGIS\Lab2Data\PRISM_ppt_30yr_normal_4kmM4_all_bil.zip'
with open(zip_path, 'wb') as file:
    file.write(zip_response.content)
    
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(directory)

In [7]:
gdb= 'C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\Lab2_Notebooks.gdb'
mosaic=arcpy.management.CreateMosaicDataset(gdb, 'mosaic', arcpy.SpatialReference(4269))

In [8]:
arcpy.management.AddRastersToMosaicDataset(mosaic, "Raster Dataset", 'C:\Mac\Home\Documents\ArcGIS\Lab2Data')

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\Lab2_Notebooks\\Lab2_Notebooks.gdb\\mosaic'>

In [9]:
arcpy.management.CalculateField(
    in_table=r"mosaic\Footprint",
    field="Variable",
    expression='"precipitation"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'mosaic\\Footprint'>

In [10]:
arcpy.management.CalculateField(
    in_table=r"mosaic\Footprint",
    field="Timestamp",
    expression="DateAdd(Date(1990,12,1), $feature.OBJECTID-1, 'month')",
    expression_type="ARCADE",
    code_block="",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'mosaic\\Footprint'>

In [11]:
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset="mosaic",
    variable_field="Variable",
    dimension_fields="Timestamp # #",
    variable_desc_units=None,
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'mosaic'>

In [12]:
mdraster=arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster="mosaic",
    out_multidimensional_raster_layer="C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\mosaic_MultidimLayer",
    variables="precipitation",
    dimension_def="ALL",

)

In [14]:
arcpy.management.CopyRaster(
    in_raster=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\mosaic_MultidimLayer",
    out_rasterdataset=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\mosaic_MultidimLayer_CopyRaster.crf",
    config_keyword="",
    background_value=None,
    nodata_value="",
    onebit_to_eightbit="NONE",
    colormap_to_RGB="NONE",
    pixel_type="",
    scale_pixel_value="NONE",
    RGB_to_Colormap="NONE",
    format="CRF",
    transform="NONE",
    process_as_multidimensional="ALL_SLICES",
    build_multidimensional_transpose="NO_TRANSPOSE"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\Lab2_Notebooks\\mosaic_MultidimLayer_CopyRaster.crf'>

In [15]:
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster="C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\mosaic_MultidimLayer_CopyRaster.crf",
    output_cube=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab2_Notebooks\prism_stc.nc",
    fill_empty_bins="ZEROS"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\Lab2_Notebooks\\prism_stc.nc'>